## 합성곱 구현

In [1]:
import numpy as np

w = np.array([2, 1, 5, 3])
x = np.array([2, 8, 3, 7, 1, 2, 0, 4, 5])

w_r = np.flip(w)    # 배열 뒤집기
print(w_r)

[3 5 1 2]


In [2]:
for i in range(6):
    print(np.dot(x[i:i + 4], w_r))

63
48
49
28
21
20


In [3]:
# scipy 이용한 합성곱 구현
from scipy.signal import convolve

convolve(x, w, mode='valid')

array([63, 48, 49, 28, 21, 20])

In [4]:
# 교차상관과의 차이: 합성곱과 달리 배열을 뒤집지 않음
from scipy.signal import correlate

correlate(x, w, mode='valid')

array([48, 57, 24, 25, 16, 39])

## 패딩(padding)과 스트라이드(stride)
* 패딩: 원본 배열의 양 끝에 빈 원소 추가
* 스트라이드: 미끄러지는 배열의 간격을 조절하는 것
* 개념 적용에 따라 valid padding, full padding, same padding 으로 부름

In [ ]:
# valid padding: 원본 배열에 패딩 추가하지 않고 미끄러지는 배열이 원본 배열 끝으로 갈 때까지 교차 상관 수행
# valid padding 결과로 얻는 배열 크기는 원본 배열보다 항상 작음

In [5]:
# full padding: 원본 배열의 모든 요소가 동일하게 연산에 참여하는 패딩 방식
correlate(x, w, mode='full')

# zero padding: 원본 배열 원소가 연산에 동일하게 참여하도록 하기 위해 원본 배열 양 끝에 가상의 원소를 추가하는데, 이때 0을 가상의 원소로 사용하므로 zero padding 이라 함

array([ 6, 34, 51, 48, 57, 24, 25, 16, 39, 29, 13, 10])

In [6]:
# same padding: 출력 배열의 길이를 원본 배열의 길이와 동일하게 제로 패딩을 추가함
correlate(x, w, mode='same')

array([34, 51, 48, 57, 24, 25, 16, 39, 29])

* 스트라이드: 미끄러지는 간격 조정
    * 합성곱 신경망을 만들 때는 스트라이드를 일반적으로 1로 지정

## 2차원 배열에서의 합성곱

In [7]:
# 합성곱 신경망은 대부분 2차원 배열에 대한 합성곱을 사용함

# 2차원 원본배열
x = np.array([
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 9]
])
# 미끄러지는 배열
w = np.array([
    [2, 0], [0, 0]
])

from scipy.signal import correlate2d

correlate2d(x, w, mode='valid')

array([[ 2,  4],
       [ 8, 10]])

In [8]:
correlate2d(x, w, mode='same')

array([[ 2,  4,  6],
       [ 8, 10, 12],
       [14, 16, 18]])

## Tensorflow 를 이용한 합성곱: `conv2d()`

In [9]:
import tensorflow as tf

# 2차원 배열을 4차원 배열로 바꾸어 합성곱 수행
# 입력 자료형을 실수(float)로 바꾸기
# 배치와 컬러 채널: 1
x_4d = x.astype(np.float).reshape(1, 3, 3, 1)
w_4d = w.reshape(2, 2, 1, 1)

c_out = tf.nn.conv2d(x_4d, w_4d, strides=1, padding='SAME')
c_out.numpy().reshape(3, 3)

# `conv2d()` 함수에 전달되는 매개변수 값은 4차원 배열임!

array([[ 2.,  4.,  6.],
       [ 8., 10., 12.],
       [14., 16., 18.]])

## 풀링 연산

* 합성곱 신경망 구조
    * 합성곱층: 합성곱이 일어나는 층
    * 풀링층: 풀링이 일어나는 층
* 특성 맵(feature map): 합성곱층과 풀링층에서 만들어진 결과
* 풀링: 특성 맵을 스캔하며 최댓값을 고르거나 평균값을 계산하는 것

* 최대 풀링(max pooling): 최댓값을 고르는 방식
    * 풀링 영역의 크기: 2 x 2
    * 스트라이드: 풀링의 한 모서리 크기로 지정(여기서는 2) - 풀링 영역이 겹치지 않도록 함
    * 가장 큰 특징을 유지시킴 - 이미지 분류에 알맞음
* 평균 풀링(average pooling): 풀링 영역의 평균값 계산
    * 합성곱층을 통과하는 특징을 희석시킬 가능성이 높음
    * 가장 큰 특성의 값을 상쇄

In [11]:
x = np.array([
    [1, 2, 3, 4],
    [5, 6, 7, 8],
    [9, 10, 11, 12],
    [13, 14, 15, 16]
])
x = x.reshape(1, 4, 4, 1)

p_out = tf.nn.max_pool2d(x, ksize=2, strides=2, padding='VALID')
p_out.numpy().reshape(2, 2)

array([[ 6.,  8.],
       [14., 16.]], dtype=float32)